# Малютин Павел, ИТПЗ РАН
 

In [ ]:
import pandas as pd
import math 
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from numba import jit, njit
from sklearn.linear_model import LinearRegression

# b-value calculating

In [ ]:
'''чтение файла из директории проги
и устранение выбросов значений магнитуды округлением до 1 знака пз.'''
my_cat = pd.read_csv('./catalogues/DOUBLE_PREC_DD (2).csv', sep = ';') 
#my_cat = my_cat.loc[my_cat.YEAR < 1986]
my_cat = my_cat.loc[my_cat.latitude > 38.7751667]
my_cat = my_cat.loc[my_cat.latitude < 38.8081667]
my_cat = my_cat.loc[my_cat.longitude > -122.778]
my_cat = my_cat.loc[my_cat.longitude < -122.733]
my_cat = my_cat.reset_index(drop = True)
my_cat.mag = round(my_cat.mag, 1)  
my_cat

In [ ]:
from pathlib import Path  
filepath = Path('./results/my_cat.csv')
my_cat.to_csv(filepath)

In [ ]:
'''Следующий кусок вычисляет необходимые для построения графика повторяемости EQ параметры
а именно: N_tot - суммарное количество землетрясений
a - Series, состоящая из значений lg(N/N_tot) и соотвествующих магнитуд
и, собственно строит график ПЗ для оценки справедливости Г-Р.'''
N_tot = my_cat.shape
f = lambda x: math.log10(x/N_tot[0])
a = my_cat.mag.value_counts().apply(f)
a.plot(style='.')
plt.grid()
plt.xlabel("M")
plt.ylabel("logN")
plt.suptitle('График повторяемости землетрясений', 
             fontsize=14, fontweight='bold')
 # расчет b-value по формуле Aki
# mag - СТОЛБЕЦ С МАГНИТУДОЙ В ДАННОЙ csv


### Необходимо выбрать представительную магнитуду

In [ ]:
M_0 = float(input())
my_cat = my_cat.loc[my_cat['mag'] > M_0]
my_cat = my_cat.reset_index(drop = True)
b_value = 1 / (my_cat['mag'].mean() - M_0) * (1/ math.log(10))
# расчет b-value по формуле Aki
# mag - СТОЛБЕЦ С МАГНИТУДОЙ В ДАННОЙ csv
print('\033[1m'+'b-value is:', b_value, ' ± ', b_value/math.sqrt(N_tot[0]))
my_cat
N_tot = my_cat.shape

# Fractal dimension

## Непосредственный расчёт фрактальной размерности

In [ ]:
Pi2 = math.pi/2 # тупо пи-на-два
RAD = 0.01745329 # DGR -> RAD
FLAT = 0.993231 # поправка на эллипсоидноть
'''создание numpy-array для более быстрого обращения [к памяти]
в цикле к соответствующим значениям широты и долготы'''
latitude = (my_cat.latitude.to_numpy())
longitude = (my_cat.longitude.to_numpy())
depth = my_cat.depth.to_numpy()

In [ ]:
PI_2 = np.array([[Pi2]*N_tot[0] for i in range(N_tot[0])])
col = PI_2 - np.arctan(FLAT*np.tan(RAD*(np.array([latitude for i in range(N_tot[0])]))))
dlon = RAD*np.transpose(np.array([longitude for i in range (N_tot[0])])) - RAD*(np.array([longitude for i in range (N_tot[0])]))
delr = np.arccos(np.multiply(np.multiply(np.sin(col),np.sin(np.transpose(col))),np.cos(dlon)) + np.multiply(np.cos(col),np.cos(np.transpose(col))))
colat = PI_2 - (0.5*(np.transpose(RAD*(np.array([latitude for i in range(N_tot[0])])))+RAD*(np.array([latitude for i in range(N_tot[0])])))) 
kek =  np.sqrt(np.power(np.multiply(6371.227*delr,(np.array([[1]*N_tot[0] for i in range(N_tot[0])]) + 0.00337853*(np.array([[1./3]*N_tot[0] for i in range(N_tot[0])]) -np.power(np.cos(colat),2)))),2) + np.power(np.transpose(np.array([depth for i in range(N_tot[0])]))-(np.array([depth for i in range(N_tot[0])])),2))

kek = np.tril(kek, k=0)
kek[kek == 0] = float('inf')

In [ ]:
la, fi, d, PI_2, col, dlon, sin_col, cos_col, delr, colat = 0,0,0,0,0,0,0,0,0,0
latitude, longitude, depth = 0,0,0

In [ ]:
fractalka = pd.DataFrame(dtype='float')  #ДФ с lg(N/N_0) и lg(A)
X = np.linspace(0.1,1.4,24) #задает равномерно распределенный массив возможных расстояний м/у ЗТ
element_count = 0
total_pairs = N_tot[0]*(N_tot[0]-1)/2    #общее количество пар землетрясений 
for i in tqdm(X):
    element_count = (kek < i).sum()
    fractalka = pd.concat([fractalka, pd.DataFrame ([math.log10(element_count/total_pairs)],[math.log10(i)])])
    element_count = 0

In [ ]:
fractalka.plot(style='.')
plt.suptitle('График зав-ти логарифма отн. числа пар землетрясений' '\nпри заданном расстоянии от логарифма этого расстояния', 
             fontsize=14, fontweight='bold')
plt.grid()
plt.xlabel("log(A)")
plt.ylabel("log(N/N_0)")
X = fractalka.index.to_numpy().reshape(-1, 1)
Y = fractalka[0].to_numpy().reshape(-1, 1)
linear_model_d = LinearRegression()
linear_model_d.fit(X, Y)
d_f = linear_model_d.coef_[0][0] # фрактальная размерность
print('\033[1m' + 'Fractal dimension is', d_f)

In [ ]:
kek[2,1]

# Функция близости

In [ ]:
print("b-value is               ", round(b_value,3))
print("d_f is                   ", round(d_f,3))
print("Minimal magnitude M_0 is ", M_0)

### Функция для рассчёта времени между зт

In [ ]:
n_ = np.zeros((N_tot[0],N_tot[0])) #numpy-array - будущая матрица расстояний м/у событиями
'''нет смысла задавать inf, потому что потом все равно будут перемножения элементов матриц'''

In [ ]:
TIME_IN_YEARS = np.array([my_cat.YEAR.to_numpy()*31536000 for i in range(N_tot[0])]) + np.array([(my_cat.MONTH.to_numpy())*2629746 for i in range(N_tot[0])])+ np.array([my_cat.DAY.to_numpy()*86400 for i in range(N_tot[0])])+ np.array([my_cat.HOUR.to_numpy()*3600 for i in range(N_tot[0])])+ np.array([my_cat.MINUTE.to_numpy()*60 for i in range(N_tot[0])])+ np.array([my_cat.SECOND.to_numpy() for i in range(N_tot[0])])

In [ ]:
delta_TIME_IN_YEARS = np.transpose(TIME_IN_YEARS) - TIME_IN_YEARS 
TIME_IN_YEARS = 0
delta_TIME_IN_YEARS.shape
delta_TIME_IN_YEARS = np.tril(delta_TIME_IN_YEARS, k = 0)
delta_TIME_IN_YEARS[delta_TIME_IN_YEARS < 0] = abs(delta_TIME_IN_YEARS[np.where(delta_TIME_IN_YEARS < 0)])

In [ ]:
delta_TIME_IN_YEARS[2,1], kek[2,1]

In [ ]:
mag = np.array([my_cat.mag.to_numpy() for i in range(N_tot[0])])
#mag = np.tril(np.transpose(mag), k = 0)
mag = np.tril((mag), k = 0)

In [ ]:
na = np.multiply(np.multiply(delta_TIME_IN_YEARS[2,1],np.power(kek[2,1], d_f)),np.power(10,(-b_value)*mag[2,1]))
na

In [ ]:
n_ = np.multiply(np.multiply(delta_TIME_IN_YEARS,np.power(kek, d_f)),np.power(10,(-b_value)*mag))
n_ = np.tril(n_, k=0)
n_[n_ == 0] = float('inf') 
n_ = np.nan_to_num(n_, nan=float('inf'), posinf=float('inf'))

In [ ]:
n_[3,]

In [ ]:
kekus = np.load('n_.npy')

In [ ]:
kekus[3,]

In [ ]:
N = np.array([0.] * N_tot[0]) #Минимальное расстояние до соседа
D = np.array([0.] * N_tot[0]) #Номер события, до которого расстояние минимально (в перспективе - ближайший сосед)
#Поиск соответствующих расстояний и номеров
for i in range(len(n_)-1):
    D[i] = round(math.log10(min(n_[i])),1)
    #N[i] = n_[:][i].index(min(n_[i][:]))
resultate = pd.DataFrame({'Closest_neigh':D, 'Index_o_CN':N})

In [ ]:
resultate
from pathlib import Path  
filepath = Path('./results/nearest_dist_n_index.csv')
resultate.to_csv(filepath)

In [ ]:
Dist_to_Nearest = resultate.Closest_neigh.value_counts(normalize=True)
from pathlib import Path  
filepath = Path('./results/nearest_dist.csv')
Dist_to_Nearest.to_csv(filepath)

In [ ]:
part_cat = resultate.loc[resultate.index < 4000]
part_cat.Closest_neigh.mean()

In [ ]:
resultate = resultate.drop(index=0)

In [ ]:
resultate

##### 